In [ ]:
try:
    import spacy
except ImportError:
    !pip install -q spacy

try:
    import nltk
except ImportError:
    !pip install -q nltk

import sys
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_input(text):
    text = text.lower()
    text = re.sub(r'(.)\1{2,}', r'\1\1', text)
    tokens = nltk.word_tokenize(text)
    cleaned = [lemmatizer.lemmatize(w) for w in tokens if w not in stop_words]
    return " ".join(cleaned)

import spacy
try:
    nlp = spacy.load('en_core_web_md')
except:
    !python -m spacy download en_core_web_md
    nlp = spacy.load('en_core_web_md')

intents = {
    'greeting': {
        'examples': [
            "hello", "hi", "hey", "good morning", "greetings",
            "good afternoon", "gm", "yo"
        ],
        'response': "Hi! How can I help you today?"
    },

    'hours': {
        'examples': [
            "business hours", "what is the working hours",
            "opening hours", "customer service", "working period",
            "customer service available", "customer service timing"
        ],
        'response': "We’re available from 9 AM to 6 PM, Monday to Friday."
    },
    'returns': {
        'examples': [
            "return of product", "can i get a refund", "refund policy",
            "reimburse", "what is the return policy", "can i return the product",
            "how long will it take to return", "how to return the product"
        ],
        'response': "You can return items within 30 days with your receipt."
    },
    'shipping': {
        'examples': [
            "transportation cost", "shipping cost", "how long will it take to ship",
            "what is the delivery charge", "is the delivery free",
            "when will the product be delivered", "when will the ship arrive",
            "how much for shipment", "product delivery", "delivery details"
        ],
        'response': "Free shipping on orders over Rs.1000. Delivery takes 3-5 days."
    },
    'order_cancellation': {
        'examples': [
            "order cancellation", "cancel", "cancel my order", "cancel policy",
            "how to cancel order", "can i cancel my order",
            "order cancellation details", "order cancellation policy"
        ],
        'response': "You can cancel within 1 hour of ordering."
    },
    'payment_methods': {
        'examples': [
            "can i pay using gpay", "is cash payment available",
            "what are the payment methods", "payment methods", "money", "credit card"
        ],
        'response': "We accept credit cards, Gpay, Cash, and Apple Pay."
    },
    'discounts': {
        'examples': [
            "any offers today", "discounts available", "is there a promo code",
            "sale", "special deals", "current promotions",
            "are there any discounts", "coupon code", "give me a discount"
        ],
        'response': "Check our latest deals on the homepage. Use code NEXUS for 10% off on your first order!"
    },
    'complaint': {
        'examples': [
            "I want to file a complaint", "bad service", "product is defective",
            "not satisfied", "very disappointed", "unhappy with the product",
            "want to report an issue", "raise a complaint", "I have a complaint"
        ],
        'response': "We’re really sorry to hear that. Please provide more details or email us at complaints@nexus.com with your order ID."
    },
    'thanks': {
        'examples': [
            "thank you", "thanks", "thank you very much", "thx", "ty"
        ],
        'response': "You’re welcome! Let me know if you need anything else."
    },
    'help': {
        'examples': [
            "help", "what can I ask", "how can you assist me",
            "what do you do", "can you help me", "assistance"
        ],
        'response': "I can help you with topics like: \n- Shipping \n- Returns & Refunds \n- Payment Methods \n- Contact Team \n- Order Cancellations \nJust ask about any of these!"
    },
    'contact': {
        'examples': [
            "how to contact", "customer care number", "email support",
            "support team", "how can i reach support team"
        ],
        'response': "You can contact us at: \n support@nexus.com \n call - +91-1234567890"
    }
}

intent_docs = {}
for intent_name, intent_data in intents.items():
    docs = [nlp(example) for example in intent_data['examples']]
    intent_docs[intent_name] = docs

def get_best_intent(user_input):
    user_input_cleaned = clean_input(user_input)
    user_doc = nlp(user_input_cleaned)
    max_similarity = 0
    best_intent = None

    for intent_name, docs in intent_docs.items():
        for doc in docs:
            similarity = user_doc.similarity(doc)
            if similarity > max_similarity:
                max_similarity = similarity
                best_intent = intent_name
    if max_similarity < 0.6:
        return None
    return best_intent

def Nexus():
    print("Nexus - Customer Service Chatbot")
    print("Type 'exit' to quit.")
    print("Type 'help' for help")
    print("Ask me anything about returns, shipping, hours, payments, and more.\n")

    user_name = input("What is your name? ")
    print(f"Hello, {user_name}! How can I assist you today?")

    while True:
        user_input = input("You: ").strip()
        if user_input.lower() in ('exit', 'quit'):
            print("Nexus: Thank you for chatting. Have a great day!")
            break
        if not user_input:
            print("Nexus: Please enter a message.")
            continue

        intent = get_best_intent(user_input)
        if intent is None:
            print("Nexus: I'm sorry, I didn't understand that. Could you please rephrase or ask about something else?")
        else:
            print(f"Nexus: {intents[intent]['response']}")

if __name__ == "__main__":
    Nexus()